In [83]:
import cv2
import numpy as np

In [84]:
lena=cv2.imread("./Lena.jpg", 0)

cv2.imshow("lena", lena)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [85]:
def gaussianFuntion(x,y,sigma):
    return (1/(2*np.pi*sigma**2))*np.exp(-(x**2 + y**2) /(sigma**2))
    

In [86]:
def gaussianFunction(x, y, sigma):
    return (1/(2 * np.pi * sigma**2))*np.exp(-(x**2 + y**2)/(2 * sigma**2))

def gaussianKernel_x(size, sigma):
    k=size//2
    kernel = np.zeros((size, size), dtype=np.float32)
    for i in range(size):
        for j in range(size):
            x = i - k
            y = j - k
            gaussValue = gaussianFunction(x, y, sigma)
            kernel[i, j] = -(x / sigma**2) * gaussValue
    kernel /= np.sum(kernel)
    return kernel

def gaussianKernel_y(size, sigma):
    k = size // 2
    kernel = np.zeros((size, size), dtype=np.float32)
    for i in range(size):
        for j in range(size):
            x = i-k
            y = j-k
            gaussValue = gaussianFunction(x, y, sigma)
            kernel[i, j] = -(y / sigma**2) * gaussValue
    kernel /= np.sum(kernel)
    return kernel

gaussian_kernel_x=gaussianKernel_x(7, 1)
gaussian_kernel_y=gaussianKernel_y(7, 1)

print(gaussian_kernel_x)

print(gaussian_kernel_y)


[[-1.2673601e+04 -1.5439606e+05 -6.9195519e+05 -1.1408411e+06
  -6.9195519e+05 -1.5439606e+05 -1.2673601e+04]
 [-1.0293071e+05 -1.2539528e+06 -5.6198265e+06 -9.2655280e+06
  -5.6198265e+06 -1.2539528e+06 -1.0293071e+05]
 [-2.3065172e+05 -2.8099132e+06 -1.2593158e+07 -2.0762606e+07
  -1.2593158e+07 -2.8099132e+06 -2.3065172e+05]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 2.3065172e+05  2.8099132e+06  1.2593158e+07  2.0762606e+07
   1.2593158e+07  2.8099132e+06  2.3065172e+05]
 [ 1.0293071e+05  1.2539528e+06  5.6198265e+06  9.2655280e+06
   5.6198265e+06  1.2539528e+06  1.0293071e+05]
 [ 1.2673601e+04  1.5439606e+05  6.9195519e+05  1.1408411e+06
   6.9195519e+05  1.5439606e+05  1.2673601e+04]]
[[-1.2673601e+04 -1.0293071e+05 -2.3065172e+05  0.0000000e+00
   2.3065172e+05  1.0293071e+05  1.2673601e+04]
 [-1.5439606e+05 -1.2539528e+06 -2.8099132e+06  0.0000000e+00
   2.8099132e+06  1.2539528e+06  1.5439606e+05]
 [-6.9195

In [87]:
def convolve2D(image : np.array ,  kernel : np.array) -> np.array:
    kernel = np.flip(kernel)
    ih, iw = image.shape
    kh , kw = kernel.shape
    border_size = kh//2
    img_bordered = cv2.copyMakeBorder(image, border_size, border_size, border_size, border_size, cv2.BORDER_CONSTANT)
    bh,bw = img_bordered.shape
    result = np.zeros((ih, iw), dtype=np.float32)
    for i in range(ih):
        for j in range(iw):
            region = img_bordered[i:i+kh ,j:j+kw]
            mul = np.multiply(region,kernel)
            val = np.sum(mul)
            result[i,j] = val  
    return result 

In [88]:
convo_x = convolve2D(lena, gaussian_kernel_x)
norm_x = np.round(cv2.normalize(convo_x, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

In [89]:
cv2.imshow("Convo_X", convo_x)
cv2.imshow("norm_x", norm_x)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [90]:
convo_y = convolve2D(lena, gaussian_kernel_y)
norm_y = np.round(cv2.normalize(convo_y, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

In [91]:
cv2.imshow("Convo_y", convo_y)
cv2.imshow("norm_y", norm_y)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [92]:
gradient_mag = cv2.magnitude(convo_x.astype(np.float32), convo_y.astype(np.float32) )

grad_norm = np.round(cv2.normalize(gradient_mag, None, 0, 255, cv2.NORM_MINMAX)).astype(np.uint8)

cv2.imshow("Grad", gradient_mag)
cv2.imshow("Grad Norm", grad_norm)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [93]:
threshold_value=50
_, binary_edges = cv2.threshold(grad_norm, threshold_value, 255, cv2.THRESH_BINARY)

cv2.imshow("cv2 threshold", binary_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [94]:
def double_threshold(image, low_thresh, high_thresh):
    output = np.zeros_like(image, dtype=np.uint8)
    output[image>=high_thresh]=255
    output[image<low_thresh]=0
    mask =(image >=low_thresh)&(image<high_thresh)
    output[mask] = 128

    return output
    

In [95]:
double_t = double_threshold(grad_norm, 50, 150)

cv2.imshow("double threshold threshold", double_t)
cv2.waitKey(0)
cv2.destroyAllWindows()